## References

* [RADIATIVE VIEW FACTORS](http://webserver.dmt.upm.es/~isidoro/tc3/Radiation%20View%20factors.pdf)
* [sphere ambient occlusion](http://www.iquilezles.org/www/articles/sphereao/sphereao.htm)